In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [2]:
print("length of dataset in characters:", len(text))

length of dataset in characters: 1115394


In [3]:
## implementing the video series

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size =  len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [7]:
print(encode("Hi there"))

[20, 47, 1, 58, 46, 43, 56, 43]


In [8]:
print(decode(encode("Hi there")))

Hi there


In [9]:
import torch

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [11]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [12]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

In [15]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [16]:
torch.manual_seed(10)

In [17]:
batch_size=4
block_size=8

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # randomly sampling batch from dataset
    x = torch.stack([data[i:i+block_size] for i in ix]) # getting the training data 
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # getting the labels 
    x, y = x.to(device), y.to(device)
    return x, y

In [20]:
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)

inputs:
torch.Size([4, 8])
tensor([[47, 52, 45, 51, 39, 52, 10,  0],
        [ 0,  0, 19, 24, 27, 33, 15, 17],
        [ 1, 53, 56,  1, 39,  1, 60, 43],
        [59, 52, 58, 56, 63,  5, 57,  1]], device='cuda:0')


In [21]:
yb

tensor([[52, 45, 51, 39, 52, 10,  0, 35],
        [ 0, 19, 24, 27, 33, 15, 17, 31],
        [53, 56,  1, 39,  1, 60, 43, 56],
        [52, 58, 56, 63,  5, 57,  1, 40]], device='cuda:0')

In [22]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension 
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [47] the target: 52
when input is [47, 52] the target: 45
when input is [47, 52, 45] the target: 51
when input is [47, 52, 45, 51] the target: 39
when input is [47, 52, 45, 51, 39] the target: 52
when input is [47, 52, 45, 51, 39, 52] the target: 10
when input is [47, 52, 45, 51, 39, 52, 10] the target: 0
when input is [47, 52, 45, 51, 39, 52, 10, 0] the target: 35
when input is [0] the target: 0
when input is [0, 0] the target: 19
when input is [0, 0, 19] the target: 24
when input is [0, 0, 19, 24] the target: 27
when input is [0, 0, 19, 24, 27] the target: 33
when input is [0, 0, 19, 24, 27, 33] the target: 15
when input is [0, 0, 19, 24, 27, 33, 15] the target: 17
when input is [0, 0, 19, 24, 27, 33, 15, 17] the target: 31
when input is [1] the target: 53
when input is [1, 53] the target: 56
when input is [1, 53, 56] the target: 1
when input is [1, 53, 56, 1] the target: 39
when input is [1, 53, 56, 1, 39] the target: 1
when input is [1, 53, 56, 1, 39, 1] the target: 6

## creating BIgram Language model

In [44]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1)

In [24]:
class BigramLanguagemodel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None 
        else:
            B, T, C = logits.shape # C is the vocab size here
            #print(B,T, C)
            logits = logits.view(B*T, C) # torch takes in batch first 
            target = targets.view(B*T)
            loss = F.cross_entropy(logits, target)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            
            logits = logits[:, -1, :] # focus only on the last time step
            probs = F.softmax(logits, dim=-1) # (B,C)
            
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            
            idx = torch.cat((idx, idx_next), dim=1) # B, T+1
            
        return idx

In [25]:
m = BigramLanguagemodel(vocab_size)

In [26]:
logits, loss= m(xb, yb)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
loss

In [27]:

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


;YoSZRqyvP!HklLni3liA&nqtwip$Uw$HH$,py-AxPK
CFr$nEIkMKVdIgmvPeH:OKL?SSoN
fvGJbyqE3B3gVByqIMowzS3gaGn


In [28]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [29]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

## The mathematical trick in self-attention

In [45]:
B, T, C = 4, 8, 2

In [46]:
x = torch.randn(B, T, C)

In [47]:
x.shape

torch.Size([4, 8, 2])

In [48]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [49]:
x[0]

tensor([[-1.5256, -0.7502],
        [-0.6540, -1.6095],
        [-0.1002, -0.6092],
        [-0.9798, -1.6091],
        [-0.7121,  0.3037],
        [-0.7773, -0.2515],
        [-0.2223,  1.6871],
        [ 0.2284,  0.4676]])

In [50]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()

In [51]:
b

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

In [52]:
c = a@b

In [53]:
c

tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])

In [54]:
a = torch.tril(a)


In [55]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B, T, C) --------> (B,T,C)

In [56]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [57]:
# version 3 use softmax

In [58]:
tril = torch.tril(torch.ones(T,T))
print(tril)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


In [59]:
wei = torch.zeros((T, T))
print(wei)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [60]:
wei = wei.masked_fill(tril==0, float('-inf'))

In [61]:
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [62]:
wei = F.softmax(wei, dim=-1)

In [63]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [64]:
xbow3 = wei @ x


In [65]:
##version self-attention

In [66]:
head_size = 16
B, T, C = 4,8,32
x = torch.randn(B, T, C)

In [67]:
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

In [68]:
k = key(x)
q = query(x)

In [69]:
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16 , T) ----> (B,T,T)

In [70]:
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

In [71]:
out.shape

torch.Size([4, 8, 16])

In [72]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6662, 0.3338, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7655, 0.0461, 0.1884, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1020, 0.3775, 0.0748, 0.4458, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2204, 0.1272, 0.1525, 0.1016, 0.3983, 0.0000, 0.0000, 0.0000],
        [0.3641, 0.0901, 0.1252, 0.1107, 0.2432, 0.0667, 0.0000, 0.0000],
        [0.2401, 0.2102, 0.0899, 0.0854, 0.0582, 0.1407, 0.1755, 0.0000],
        [0.0094, 0.0462, 0.0066, 0.1216, 0.0506, 0.0098, 0.1085, 0.6472]],
       grad_fn=<SelectBackward0>)

In [73]:
class LayerNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]


In [74]:
torch.manual_seed(10)

In [75]:
module = LayerNorm1d(100)
x = torch.randn(32,100)
x = module(x)


In [76]:
x

tensor([[-0.7069, -0.4557, -0.7159,  ...,  0.1775, -0.1662, -0.0653],
        [-0.5106, -1.6074, -0.1920,  ..., -1.1063,  0.4444,  0.3785],
        [ 0.2775, -0.2590, -0.9874,  ..., -2.6589, -0.8894, -0.0554],
        ...,
        [-1.5867,  0.1589,  0.2769,  ...,  1.0696,  0.8219, -0.7920],
        [ 0.2357, -0.5743, -0.4427,  ..., -0.1657,  0.1989,  0.2594],
        [-0.2638,  0.7837,  0.0594,  ..., -0.2206,  0.6580, -0.1966]])

In [77]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(-0.0983), tensor(0.8398))

In [78]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(2.3842e-08), tensor(1.0000))

## GPT

In [79]:
batch_size = 16

In [80]:
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [81]:
torch.manual_seed(12)

In [82]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logit, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [83]:
torch.tril(torch.ones(block_size, block_size))

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [84]:
test_shape = torch.randn((2,3, 3))
print(test_shape)
print(test_shape.transpose(-2,-1))

tensor([[[-0.1320, -0.1254, -0.1610],
         [-0.8372, -1.2945, -0.2623],
         [-0.5353,  1.3466, -0.6958]],

        [[-0.0522,  0.4924, -0.6921],
         [-0.6944, -0.2125,  0.2362],
         [-1.2438,  1.4400,  0.7946]]])
tensor([[[-0.1320, -0.8372, -0.5353],
         [-0.1254, -1.2945,  1.3466],
         [-0.1610, -0.2623, -0.6958]],

        [[-0.0522, -0.6944, -1.2438],
         [ 0.4924, -0.2125,  1.4400],
         [-0.6921,  0.2362,  0.7946]]])


In [85]:
class Head(nn.Module):
    """one head of self.attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    
    def forward(self, x, debug=False):
        B, T, C = x.shape
        k = self.key(x) 
        q = self.query(x)
        
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        if debug: print(wei)
        
        if debug: print(wei.shape)
        
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        out = wei @ v
        return out 
                    

In [86]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [87]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

In [88]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [89]:
class gpt_model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.ln_head = nn.LayerNorm(n_embed, vocab_size)
        
    def forward(self, idx, targets=None):
        B,T = idx.shape
        tok_embed = self.token_embedding_table(idx)
        pos_embed = self.position_embedding_table(torch.arange(T, device=device)) #T,C 
        x = tok_embed + pos_embed
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.ln_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
            

In [90]:
from tqdm import tqdm

In [91]:
model = gpt_model()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in tqdm(range(max_iters)):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.205632 M parameters


  0%|                                                                                         | 0/5000 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
